# Phoneme to Grapheme Conversion with a Recurrent Generative Model 
This project will discuss...

In [194]:
import torch
import torch.nn as nn
import random
import torch.optim as optim
from torch.utils.data import Dataset
import pandas as pd

# known phonemes/graphemes
phonemes = ['0', '1', 'a', 'ä', 'ɑ', 'ɒ', 'æ', 'b', 'ḇ', 'β', 'c', 'č', 'ɔ', 'ɕ', 'ç', 'd', 'ḏ', 'ḍ', 'ð', 'e', 'ə', 'ɚ', 'ɛ', 'ɝ', 'f', 'g', 'ḡ', 'h', 'ʰ', 'ḥ', 'ḫ', 'ẖ', 'i', 'ɪ', 'ỉ', 'ɨ', 'j', 'ʲ', 'ǰ', 'k', 'ḳ', 'ḵ', 'l', 'ḷ', 'ɬ', 'ɫ', 'm', 'n', 'ŋ', 'ṇ', 'ɲ', 'ɴ', 'o', 'ŏ', 'ɸ', 'θ', 'p', 'p', '̅', 'þ', 'q', 'r', 'ɹ', 'ɾ', 'ʀ', 'ʁ', 'ṛ', 's', 'š', 'ś', 'ṣ', 'ʃ', 't', 'ṭ', 'ṯ', 'ʨ', 't', 'ʂ', 'u', 'ʊ', 'ŭ', 'ü', 'v', 'ʌ', 'ɣ', 'w', 'ʍ', 'x', 'χ', 'y', 'ʸ', 'ʎ', 'z', 'ẓ', 'ž', 'ʒ', 'ʔ', 'ʕ']

graphemes = ['0', '1', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

# one hot encodes the word: returns an array of one hot encoded characters
def nemes_to_1_hot_seq(string, nemes="phonemes"):
    string = '0' + string + '1'
    l = phonemes if nemes == "phonemes" else graphemes
    seq = []
    for i in string:
        vec = [0] * len(l)
        vec[l.index(i)] = 1
        seq.append(vec)

    return torch.FloatTensor([seq])

def one_hot_to_nemes(arr, nemes="phonemes"):
    seq = []
    l = phonemes if nemes == "phonemes" else graphemes
    for hot in arr:
        x = torch.argmax(hot)
        seq.append(l[x])
    return seq

class P2GDataset(Dataset):
    def __init__(self, phoneme_file):
        self.data = pd.read_csv(phoneme_file, header=None)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        p, g = self.data.iloc[idx]
        return nemes_to_1_hot_seq(p, nemes = "phonemes"), nemes_to_1_hot_seq(g, nemes = "graphemes")


In [195]:
# define model architecture
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.GRU(len(phonemes), 512, 1, batch_first=True, bidirectional=False)
        
    def forward(self, x):
        # push vector through encoder
        out, h_n = self.encoder(x)

        # return context vector
        return h_n

class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.decoder = nn.GRU(len(graphemes), 512, 1, batch_first=True, bidirectional=False)
        self.fc = nn.Linear(512, len(graphemes))

    def forward(self, input, hidden_layer):
        """
        Since this function gets called once at a time rather than taking in
        a sequence of vectors, we need to pass it the last output. This will be just
        a vector of numbers that can be converted to the embedding representing that last output
        """
        out, h_n = self.decoder(input, hidden_layer)
        # print("H")
        return self.fc(h_n), h_n

class seq2seq(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        self.device = device
    
    def forward(self, in_seq, out_seq, tf_ratio=0.5):
        out_len = out_seq.shape[1]
        # storing the outputs of the sequence
        outputs = torch.zeros(out_len, 1, len(graphemes)).to(self.device)

        hidden = self.encoder(in_seq)

        out_seq = out_seq.squeeze(0)

        input = out_seq[0].unsqueeze(0).unsqueeze(0)
        
        for i in range(1, out_len):
            out, hidden = self.decoder(input, hidden)
            outputs[i] = out

            if random.random() > tf_ratio:
                # teacher forcing (make next input what the current output token should be)
                input = out_seq[i].unsqueeze(0).unsqueeze(0)
            else:
                x = input.argmax(1)[0]
                input = torch.zeros(1, 1, len(graphemes))
                input[0][0][x] = 1
                
        return outputs

    def pred_new(self, in_seq):
        hidden = self.encoder(in_seq)
        input = torch.FloatTensor(1, 1, len(graphemes))
        outs = []
        while True:
            out, hidden = self.decoder(input, hidden)
            outs.append(out)
            x = input.argmax(1)[0]
            input = torch.zeros(1, 1, len(graphemes))
            input[0][0][x] = 1
            if one_hot_to_nemes(out) == ['1']:
                break
        return outs

In [196]:
"""training"""
from torch.utils.data import DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"
EPOCHS = 100
model = seq2seq(device)
# what a beautiful architecture
print(model)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_func = nn.CrossEntropyLoss()
dataset = P2GDataset("data.csv")
dataloader = DataLoader(dataset=dataset, batch_size=1)


seq2seq(
  (encoder): Encoder(
    (encoder): GRU(98, 512, batch_first=True)
  )
  (decoder): Decoder(
    (decoder): GRU(28, 512, batch_first=True)
    (fc): Linear(in_features=512, out_features=28, bias=True)
  )
)


In [197]:

for epoch in range(100):
    for (in_seq, out_seq) in dataloader:
        in_seq = in_seq.squeeze(0)
        out_seq = out_seq.squeeze(0)
        model_output = model(in_seq, out_seq)
        model_output = model_output[1:]
        model_output = model_output.squeeze(1)
        out_seq = out_seq.squeeze(0)[1:]
        loss = loss_func(model_output, out_seq)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


In [198]:
def print_preds(path):
    dataset = P2GDataset(path)
    p, g = dataset[0]
    s = model.pred_new(p)
    print(one_hot_to_nemes(s, "graphemes"))
print_preds("data.csv")
# print(one_hot_to_graphemes(torch.FloatTensor([[3,2,1],[0,0,1],[0,0,1]])))

['d', 'a', 'u', 'g', 'h', 't', 'e', 'r', '1']
